In [12]:
from findspark import init
init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import * 
from utils import setupLogging,versionInfo,getAllConf,getInputData

In [13]:
spark = SparkSession.builder.appName("SparkTest").master("local[*]").getOrCreate()

In [14]:
setupLogging()
logger = logging.getLogger(__name__)
logger.info("Starting....")
[logger.info(s) for s in versionInfo(spark).split('\n')]
[logger.info(s) for s in getAllConf(spark).split('\n')]



2017-11-22 18:55:18,296 - __main__ - INFO - Starting....
2017-11-22 18:55:18,301 - __main__ - INFO - 
2017-11-22 18:55:18,307 - __main__ - INFO - ---------------------------------------------------------------------------------
2017-11-22 18:55:18,309 - __main__ - INFO -  Spark version: 2.1.1
2017-11-22 18:55:18,310 - __main__ - INFO -  Spark master : local[*]
2017-11-22 18:55:18,312 - __main__ - INFO -  Spark running locally? True
2017-11-22 18:55:18,313 - __main__ - INFO -  Default parallelism: 4
2017-11-22 18:55:18,317 - __main__ - INFO - ---------------------------------------------------------------------------------
2017-11-22 18:55:18,318 - __main__ - INFO - 
2017-11-22 18:55:18,346 - __main__ - INFO - key: [spark.app.id], value: [local-1511376846915]
2017-11-22 18:55:18,347 - __main__ - INFO - key: [spark.driver.port], value: [38927]
2017-11-22 18:55:18,350 - __main__ - INFO - key: [spark.rdd.compress], value: [True]
2017-11-22 18:55:18,355 - __main__ - INFO - key: [hive.metast

[None, None, None, None, None, None, None, None, None, None]

In [7]:
empSchema = StructType([ StructField("emp_no",LongType(),False),
                        StructField("birth_date",DateType(),False),
                        StructField("first_name",StringType(),True),
                        StructField("last_name",StringType(),True),
                        StructField("gender",StringType(),True),
                        StructField("hire_date",DateType(),False) ])

titlesSchema = StructType([ StructField("emp_no",LongType(),False),
                            StructField("title",StringType(),False),
                            StructField("from_date",DateType(),True),
                            StructField("to_date",DateType(),True) ])

  

In [8]:
empsRDD = spark.sparkContext.parallelize(getInputData("data/employees.json"))
empsDF  = spark.read.schema(empSchema).json(empsRDD)
empsDF.createOrReplaceTempView("employees")

titlesRDD = spark.sparkContext.parallelize(getInputData("data/titles.json"))
titlesDF  = spark.read.schema(titlesSchema).json(titlesRDD)
titlesDF.createOrReplaceTempView("titles")

title = spark.sql("select * from titles") \
             .select('emp_no','title','from_date','to_date') \
             .where("from_date <= current_date and to_date > current_date")
title.cache()        
        
emp = spark.table("employees")
emp.cache()


DataFrame[emp_no: bigint, birth_date: date, first_name: string, last_name: string, gender: string, hire_date: date]

In [9]:
result = emp.alias('employee') \
            .select('emp_no','first_name','last_name') \
            .join(title.alias('title'),emp['emp_no'] == title['emp_no']) \
            .select("employee.emp_no","employee.first_name","employee.last_name","title.title")
for row in result.collect():
    print(row)

Row(emp_no=10156, first_name='Sumali', last_name='Fargier', title='Senior Engineer')
Row(emp_no=10422, first_name='Greger', last_name='Rubsam', title='Engineer')
Row(emp_no=10871, first_name='Aram', last_name='Gilg', title='Staff')
Row(emp_no=10636, first_name='Saeko', last_name='Porotnikoff', title='Staff')
Row(emp_no=10889, first_name='Cheong', last_name='Coors', title='Senior Staff')
Row(emp_no=10014, first_name='Berni', last_name='Genin', title='Engineer')
Row(emp_no=10430, first_name='Ferdinand', last_name='Chenney', title='Senior Engineer')
Row(emp_no=10573, first_name='Tetsushi', last_name='Speel', title='Staff')
Row(emp_no=10627, first_name='Shigeu', last_name='Itzfeldt', title='Senior Staff')
Row(emp_no=10004, first_name='Chirstian', last_name='Koblick', title='Senior Engineer')
Row(emp_no=10178, first_name='Valery', last_name='Litvinov', title='Staff')
Row(emp_no=10205, first_name='Mabhin', last_name='Leijenhorst', title='Senior Engineer')
Row(emp_no=10433, first_name='Reuven

In [10]:
logger.info("Stopped.....")

2017-11-22 18:54:24,184 - __main__ - INFO - Stopped.....
